In [1]:
import pandas as pd
import numpy as np
import re
import json
pd.set_option("display.max_rows", 5)
pd.set_option("display.max_columns", None)  
pd.set_option('display.max_colwidth', 100)

In [2]:
df = pd.read_csv('books.csv', sep=";", on_bad_lines='skip')
df = df.astype(object).where(df.notna(), None)
df

,amtsave,brand,group,country,desc,discount,domain,gallery,image,insertedon,list_price,model,name,other_sellers,payment_methods_supported,productcode,selling_price,specifications,type,uniq_id,url,weight
0,57.0,Arihant Publications,Academic & Professional,IN,"{""Description""=>{""Brand""=>""Arihant Publications"", ""Product Code""=>""9789352031481_20439"", ""ISBN-1...",27%%,https://catalog.paytm.com,None,https://assetscdn.paytm.com/images/catalog/product/9/97/9789352031481_20439/0x1920/70/0.jpg,2016-05-18 08:01:18 +0000,210,4671.0,Daily Practice Problems for Theory of Equations Complex Number,None,"[""Cash On Delivery"", ""Credit Card"", ""Debit Card"", ""Net Banking"", ""Payment Protection Insurance"",...",7473745.0,153.0,"{""specifications""=>{""binding""=>""Paperback"", ""languages""=>""English"", ""publication_year""=>""2015"", ...",Bestseller Books,a71b77ad410561fbed9f31b5313cbde6,https://paytm.com/shop/p/daily-practice-problems-for-theory-of-equations-complex-number-sequence...,500.0
1,0.0,None,Academic & Professional,IN,"{""Description""=>{""Product Code""=>""9780552553520_21345"", ""ISBN-13""=>""9780552553520"", ""ISBN-10""=>""...",0%%,https://catalog.paytm.com,None,https://assetscdn.paytm.com/images/catalog/product/9/97/9780552553520_21345/0x1920/70/0.jpg,2016-05-18 08:01:18 +0000,325,439730.0,Love Lessons,None,"[""Credit Card"", ""Debit Card"", ""Net Banking"", ""Payment Protection Insurance"", ""EMI""]",51083750.0,325.0,"{""specifications""=>{""binding""=>""Paperback"", ""languages""=>""English"", ""age_wise""=>""9-12 Years"", ""p...",Bestseller Books,5ff366dfa3f6a2b397975acf37367d6b,https://paytm.com/shop/p/love-lessons-9780552553520_21345,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1502,95.0,Shroff/O'Reilly,None,IN,"{""Description""=>{""Brand""=>""Shroff/O'Reilly"", ""Product Code""=>""9789350230299_87743"", ""ISBN-13""=>""...",11%%,https://catalog.paytm.com,None,https://assetscdn.paytm.com/images/catalog/product/9/97/9789350230299_87743/0x1920/70/0.jpg,2016-05-18 08:01:18 +0000,900,6886.0,Windows 7 Annoyances,"[{""name"":""Book Place"",""id"":191287,""offer_text"":null,""offer_price"":873,""totalScore"":-873,""product...","[""Credit Card"", ""Debit Card"", ""Net Banking"", ""Payment Protection Insurance"", ""EMI""]",20570488.0,805.0,"{""specifications""=>{""binding""=>""Paperback"", ""languages""=>""Others"", ""publication_year""=>""2010"", ""...",Bestseller Books,55c09cfe8fc2d6b2afd48fd879b5e5c9,https://paytm.com/shop/p/windows-7-annoyances-9789350230299_87743,500.0
1503,710.0,Random House,None,IN,"{""Description""=>{""Brand""=>""Random House"", ""Product Code""=>""9780099474296_65496"", ""ISBN-13""=>""978...",71%%,https://catalog.paytm.com,None,https://assetscdn.paytm.com/images/catalog/product/9/97/9780099474296_65496/0x1920/70/0.jpg,2016-05-18 08:01:18 +0000,1000,4244.0,You'll See It When You Believe It,"[{""name"":""PRAKASH BOOKS INDIA PVT. LTD."",""id"":95913,""offer_text"":null,""offer_price"":292,""totalSc...","[""Cash On Delivery"", ""Credit Card"", ""Debit Card"", ""Net Banking"", ""Payment Protection Insurance"",...",31885378.0,290.0,"{""specifications""=>{""binding""=>""Paperback"", ""languages""=>""English"", ""publication_year""=>""2005 Fe...",Bestseller Books,d5729c53b885fab0f487a47c1129fe9a,https://paytm.com/shop/p/you-ll-see-it-when-you-believe-it-9780099474296_65496,196.0


In [3]:
empty_list = [None] * len(df)
d = {
    #'amtsave': empty_list.copy(), 
    'brand': empty_list.copy(), 
    'group': empty_list.copy(), 
    #'country': empty_list.copy(), 
    #'discount': empty_list.copy(), 
    #'domain': empty_list.copy(),
    #'gallery': empty_list.copy(), 
    'image': empty_list.copy(), 
    'list_price': empty_list.copy(),
    'model': empty_list.copy(), 
    'name': empty_list.copy(),
    'productcode': empty_list.copy(),
    #'selling_price': empty_list.copy(),  
    'type': empty_list.copy(), 
    'uniq_id': empty_list.copy(), 
    #'url': empty_list.copy(), 
    'weight': empty_list.copy()
}

def null_or_val(value, is_type):
    return value if type(value) == is_type else None

def null_or_to_type(value, mtype):
    try:
        return mtype(value)
    except TypeError:
        return None
    except ValueError:
        return None

def null_or_call_method(method, value, is_type):
    try:
        return method(value) if type(value) == is_type else None
    except ValueError:
        return None

def dict_unpacked(d):
    new_d = {}
    for key, value in d.items(): 
        if type(value) == dict:
               new_d.update(dict_unpacked(value))
        else:
            new_d[key] = value
    return new_d

for i, row in df.iterrows():
    #d['amtsave'][i] =       null_or_val(row['amtsave'], float) #int
    d['brand'][i] =         null_or_val(row['brand'], str)
    d['group'][i] =         null_or_val(row['group'], str)
    #d['country'][i] =       null_or_val(row['country'], str)
    #d['discount'][i] =      null_or_val(row['discount'], str) # int
    #d['domain'][i] =        null_or_val(row['domain'], str)
    #d['gallery'][i] =       null_or_val(row['gallery'], str)
    d['image'][i] =         null_or_val(row['image'], str)
    d['list_price'][i] =    null_or_to_type(row['list_price'], int) #int
    d['model'][i] =         null_or_to_type(row['model'], int) #int
    d['name'][i] =          null_or_val(row['name'], str)
    d['productcode'][i] =   null_or_to_type(row['productcode'], int) #int
    #d['selling_price'][i] = null_or_val(row['selling_price'], float)
    d['type'][i] =          null_or_val(row['type'], str)
    d['uniq_id'][i] =       null_or_val(row['uniq_id'], str)
    #d['url'][i] =           null_or_val(row['url'], str)
    d['weight'][i] =        null_or_to_type(row['weight'], int) #int

    for json_string in [row['desc'], row['specifications']]:
        if type(json_string) == str:
            json_string_true = json_string.replace("=>", ":")

            try:
                json_dict = json.loads(json_string_true)
                
                if(type(json_dict) == dict):
                    for key, value in dict_unpacked(json_dict).items():
                        if key+"_" not in d:
                            d[key+"_"] = empty_list.copy()
                        d[key+"_"][i] = value
            except json.JSONDecodeError:
                pass
    #"desc", "specifications"
new_df = pd.DataFrame(d, dtype=object)
new_df = new_df.reindex(sorted(new_df.columns.values, key=str.casefold), axis=1)
new_df.to_csv('new_books.csv', index=False)

new_df

,About the Author_,Age wise_,Age_,age_wise_,Author Info_,Author_,Authored By_,Binding_,binding_,brand,Brand_,browse_by_author_,Dimensions (L X B X H)_,Edited By_,Edition_,Estimated Arrival_,exam_,group,image,Imprint_,imprint_,Imprints_,ISBN 13_,ISBN-10_,ISBN-13_,ISBN10_,ISBN13_,isbn_13_,Language_,Languages_,languages_,list_price,model,name,No. Of Pages_,Number Of Pages_,Product Code_,Product Weight_,productcode,Publication Year_,publication_year_,Publisher Name_,Publisher name_,Publisher_,publisher_name_,Return Policy_,Series Name_,Shipping Charge_,Specialization_,specialization_,type,Type_,type_,uniq_id,University_,university_,Warranty Summary_,weight,Weight(gm)_,Weight(Gm)_,Weight_,weight_
0,None,None,None,None,"An editorial team of highly skilled professionals at Arihant, works hand in glove to ensure that...",None,Amit M Agarwal,Paperback,Paperback,Arihant Publications,Arihant Publications,None,None,None,1,6 - 9 days,JEE Main And Advanced,Academic & Professional,https://assetscdn.paytm.com/images/catalog/product/9/97/9789352031481_20439/0x1920/70/0.jpg,None,None,None,9789352031481,9352031482,9789352031481,None,None,9789352031481,English,English,English,210,4671,Daily Practice Problems for Theory of Equations Complex Number,240,None,9789352031481_20439,None,7473745,2015,2015,Arihant,None,None,Wiley India Pvt. Ltd.,Seller will accept returns within a 15 days from date of delivery of the item,None,None,None,None,Bestseller Books,None,None,a71b77ad410561fbed9f31b5313cbde6,None,None,None,500,None,None,None,None
1,None,9-12 Years,9 - 12 Years,9-12 Years,None,None,Jacqueline Wilson,Paperback,Paperback,None,None,None,None,None,None,5 - 8 days,None,Academic & Professional,https://assetscdn.paytm.com/images/catalog/product/9/97/9780552553520_21345/0x1920/70/0.jpg,Corgi Childrens,None,None,9780552553520,552553522,9780552553520,None,None,9780552553520,English,English,English,325,439730,Love Lessons,272,None,9780552553520_21345,None,51083750,2006,2006,Corgi,None,None,Corgi Childrens,Seller will accept returns within a 15 days from date of delivery of the item,None,None,None,None,Bestseller Books,None,None,5ff366dfa3f6a2b397975acf37367d6b,None,None,None,500,None,None,200 g,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1502,None,None,None,None,None,None,David Karp,Paperback,Paperback,Shroff/O'Reilly,Shroff/O'Reilly,None,None,None,None,5 - 8 days,None,None,https://assetscdn.paytm.com/images/catalog/product/9/97/9789350230299_87743/0x1920/70/0.jpg,None,None,None,9789350230299,\t9350230291,9789350230299,None,None,9789350230299,None,Others,Others,900,6886,Windows 7 Annoyances,740,None,9789350230299_87743,None,20570488,2010,2010,Shroff Publishers & Distributors Pvt. Ltd.,None,None,Shroff Publishers And Distributors Pvt.ltd,Seller will accept returns within a 15 days from date of delivery of the item,None,None,None,None,Bestseller Books,None,None,55c09cfe8fc2d6b2afd48fd879b5e5c9,None,None,None,500,None,None,None,None
1503,None,None,None,None,None,None,Wayne W Dyer,Paperback,Paperback,Random House,Random House,None,None,None,1,5 - 8 days,None,None,https://assetscdn.paytm.com/images/catalog/product/9/97/9780099474296_65496/0x1920/70/0.jpg,None,None,None,9780099474296,99474298,9780099474296,None,None,9780099474296,English,English,English,1000,4244,You'll See It When You Believe It,272,None,9780099474296_65496,None,31885378,2005 February,2005 February,Random House,None,None,Random House,Seller will accept returns within a 15 days from date of delivery of the item,None,None,None,None,Bestseller Books,None,None,d5729c53b885fab0f487a47c1129fe9a,None,None,None,196,None,None,196 gm,None


In [4]:
new_df.columns

Index(['About the Author_', 'Age wise_', 'Age_', 'age_wise_', 'Author Info_',
       'Author_', 'Authored By_', 'Binding_', 'binding_', 'brand', 'Brand_',
       'browse_by_author_', 'Dimensions (L X B X H)_', 'Edited By_',
       'Edition_', 'Estimated Arrival_', 'exam_', 'group', 'image', 'Imprint_',
       'imprint_', 'Imprints_', 'ISBN 13_', 'ISBN-10_', 'ISBN-13_', 'ISBN10_',
       'ISBN13_', 'isbn_13_', 'Language_', 'Languages_', 'languages_',
       'list_price', 'model', 'name', 'No. Of Pages_', 'Number Of Pages_',
       'Product Code_', 'Product Weight_', 'productcode', 'Publication Year_',
       'publication_year_', 'Publisher Name_', 'Publisher name_', 'Publisher_',
       'publisher_name_', 'Return Policy_', 'Series Name_', 'Shipping Charge_',
       'Specialization_', 'specialization_', 'type', 'Type_', 'type_',
       'uniq_id', 'University_', 'university_', 'Warranty Summary_', 'weight',
       'Weight(gm)_', 'Weight(Gm)_', 'Weight_', 'weight_'],
      dtype='object')

In [5]:
from ast import match_case


short_d = {
    'Author_Info': empty_list.copy(),
    'Age': empty_list.copy(),
    'Author': empty_list.copy(),
    'Binding': empty_list.copy(),
    'Brand': empty_list.copy(),
    'Dimensions (L X B X H)': empty_list.copy(),
    'Editor': empty_list.copy(),
    'Edition': empty_list.copy(),
    'Exam': empty_list.copy(),
    'Group': empty_list.copy(),
    'Image': empty_list.copy(),
    'Imprint': empty_list.copy(),
    'ISBN_10': empty_list.copy(),
    'ISBN_13': empty_list.copy(),
    'Language': empty_list.copy(),
    'Price': empty_list.copy(),
    'Model': empty_list.copy(),
    'Name': empty_list.copy(),
    'Pages': empty_list.copy(),
    'Product_Code_1': empty_list.copy(),
    'Product_Code_2': empty_list.copy(),
    'Weight_Gram': empty_list.copy(),
    'Publication_Year': empty_list.copy(),
    'Publication_Month': empty_list.copy(),
    'Publisher': empty_list.copy(),
    'Series_Name': empty_list.copy(),
    'Type': empty_list.copy(),
    'Unique_Id': empty_list.copy(),
    'University': empty_list.copy(),
}

def null_or_method(value, method):
    if value != None:
        return method(value)
    return None

def int_ignore_letters(value):
    #print(value, type(value))
    if type(value) == int:
        return value
    if type(value) == str:
        string = ''.join(c for c in value if c.isdigit())
        if string:
            return int(string)
        return None
    return int(value)  

def string_ignore_integer(value):
    return ''.join(c for c in value if c.isalpha())

float_regex = re.compile("\d+\.\d+", re.VERBOSE)

def to_gram_int(value):
    if(type(value) == int):
        return value
    weight_type = string_ignore_integer(value)
    if(weight_type in ["g", "gm"]):
        return int_ignore_letters(value)
    elif(weight_type in ["kg", "kgs"]):
        return int(float(float_regex.findall(value)[0])*1000)

for i, row in new_df.iterrows():
    year_and_month = row['Publication Year_'] or row['publication_year_']
    short_d['Publication_Year'][i] = null_or_method(year_and_month, int_ignore_letters)
    short_d['Publication_Month'][i] = null_or_method(year_and_month, string_ignore_integer)
    short_d['Price'][i] = null_or_method(row['list_price'] or row['Number Of Pages_'], int_ignore_letters)
    short_d['Pages'][i] = null_or_method(row['No. Of Pages_'] or row['Number Of Pages_'], int_ignore_letters)
    short_d['Model'][i] = null_or_method(row['model'], int_ignore_letters)
    short_d['Edition'][i] = null_or_method(row['Edition_'], int_ignore_letters)
    short_d['ISBN_10'][i] = null_or_method(row['ISBN-10_'] or row['ISBN10_'], int_ignore_letters)
    short_d['ISBN_13'][i] = null_or_method(row['ISBN-13_'] or row['isbn_13_'] or row['ISBN 13_'] or row['ISBN13_'], int_ignore_letters)
    short_d['Product_Code_2'][i] = null_or_method(row['productcode'], int_ignore_letters)
    short_d['Weight_Gram'][i] = null_or_method(row['Weight_'] or row['Product Weight_'] or row['Weight(Gm)_'] or row['Weight(gm)_'] or row['weight_'] or row['weight'], to_gram_int)
    short_d['Author_Info'][i] = row['About the Author_'] or row['Author Info_']
    short_d['Age'][i] = row['Age_'] or row['Age wise_'] or row['age_wise_']
    short_d['Author'][i] = row['Author_'] or row['Authored By_'] or row['browse_by_author_']
    short_d['Binding'][i] = row['Binding_'] or row['Binding_']
    short_d['Brand'][i] = row['brand'] or row['Brand_']
    short_d['Dimensions (L X B X H)'][i] = row['Dimensions (L X B X H)_']
    short_d['Editor'][i] = row['Edited By_']
    short_d['Exam'][i] = row['exam_']
    short_d['Group'][i] = row['group']
    short_d['Image'][i] = row['image']
    short_d['Imprint'][i] = row['Imprint_'] or row['imprint_'] or row['Imprints_']
    short_d['Language'][i] = row['Languages_'] or row['Language_'] or row['languages_']
    short_d['Name'][i] = row['name']
    short_d['Product_Code_1'][i] = row['Product Code_']
    short_d['Publisher'][i] = row['publisher_name_'] or row['Publisher Name_']  or row['Publisher_'] or row['Publisher name_']
    short_d['Series_Name'][i] = row['Series Name_']
    short_d['Type'][i] = row['type'] or row['Type_'] or row['type_']
    short_d['Unique_Id'][i] = row['uniq_id']
    short_d['University'][i] = row['University_'] or row['university_']

compact_df = pd.DataFrame(short_d, dtype=object)
compact_df = compact_df.reindex(sorted(compact_df.columns.values, key=str.casefold), axis=1)
compact_df.to_csv('compact_books.csv', index=False)
compact_df

,Age,Author,Author_Info,Binding,Brand,Dimensions (L X B X H),Edition,Editor,Exam,Group,Image,Imprint,ISBN_10,ISBN_13,Language,Model,Name,Pages,Price,Product_Code_1,Product_Code_2,Publication_Month,Publication_Year,Publisher,Series_Name,Type,Unique_Id,University,Weight_Gram
0,None,Amit M Agarwal,"An editorial team of highly skilled professionals at Arihant, works hand in glove to ensure that...",Paperback,Arihant Publications,None,1,None,JEE Main And Advanced,Academic & Professional,https://assetscdn.paytm.com/images/catalog/product/9/97/9789352031481_20439/0x1920/70/0.jpg,None,9352031482,9789352031481,English,4671,Daily Practice Problems for Theory of Equations Complex Number,240,210,9789352031481_20439,7473745,,2015,Wiley India Pvt. Ltd.,None,Bestseller Books,a71b77ad410561fbed9f31b5313cbde6,None,500
1,9 - 12 Years,Jacqueline Wilson,None,Paperback,None,None,None,None,None,Academic & Professional,https://assetscdn.paytm.com/images/catalog/product/9/97/9780552553520_21345/0x1920/70/0.jpg,Corgi Childrens,552553522,9780552553520,English,439730,Love Lessons,272,325,9780552553520_21345,51083750,,2006,Corgi Childrens,None,Bestseller Books,5ff366dfa3f6a2b397975acf37367d6b,None,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1502,None,David Karp,None,Paperback,Shroff/O'Reilly,None,None,None,None,None,https://assetscdn.paytm.com/images/catalog/product/9/97/9789350230299_87743/0x1920/70/0.jpg,None,9350230291,9789350230299,Others,6886,Windows 7 Annoyances,740,900,9789350230299_87743,20570488,,2010,Shroff Publishers And Distributors Pvt.ltd,None,Bestseller Books,55c09cfe8fc2d6b2afd48fd879b5e5c9,None,500
1503,None,Wayne W Dyer,None,Paperback,Random House,None,1,None,None,None,https://assetscdn.paytm.com/images/catalog/product/9/97/9780099474296_65496/0x1920/70/0.jpg,None,99474298,9780099474296,English,4244,You'll See It When You Believe It,272,1000,9780099474296_65496,31885378,February,2005,Random House,None,Bestseller Books,d5729c53b885fab0f487a47c1129fe9a,None,196


In [10]:
final_df = compact_df[pd.notnull(compact_df['Name'])]
final_df.to_csv('final.csv', index=False)
final_df

,Age,Author,Author_Info,Binding,Brand,Dimensions (L X B X H),Edition,Editor,Exam,Group,Image,Imprint,ISBN_10,ISBN_13,Language,Model,Name,Pages,Price,Product_Code_1,Product_Code_2,Publication_Month,Publication_Year,Publisher,Series_Name,Type,Unique_Id,University,Weight_Gram
0,None,Amit M Agarwal,"An editorial team of highly skilled professionals at Arihant, works hand in glove to ensure that...",Paperback,Arihant Publications,None,1,None,JEE Main And Advanced,Academic & Professional,https://assetscdn.paytm.com/images/catalog/product/9/97/9789352031481_20439/0x1920/70/0.jpg,None,9352031482,9789352031481,English,4671,Daily Practice Problems for Theory of Equations Complex Number,240,210,9789352031481_20439,7473745,,2015,Wiley India Pvt. Ltd.,None,Bestseller Books,a71b77ad410561fbed9f31b5313cbde6,None,500
1,9 - 12 Years,Jacqueline Wilson,None,Paperback,None,None,None,None,None,Academic & Professional,https://assetscdn.paytm.com/images/catalog/product/9/97/9780552553520_21345/0x1920/70/0.jpg,Corgi Childrens,552553522,9780552553520,English,439730,Love Lessons,272,325,9780552553520_21345,51083750,,2006,Corgi Childrens,None,Bestseller Books,5ff366dfa3f6a2b397975acf37367d6b,None,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1502,None,David Karp,None,Paperback,Shroff/O'Reilly,None,None,None,None,None,https://assetscdn.paytm.com/images/catalog/product/9/97/9789350230299_87743/0x1920/70/0.jpg,None,9350230291,9789350230299,Others,6886,Windows 7 Annoyances,740,900,9789350230299_87743,20570488,,2010,Shroff Publishers And Distributors Pvt.ltd,None,Bestseller Books,55c09cfe8fc2d6b2afd48fd879b5e5c9,None,500
1503,None,Wayne W Dyer,None,Paperback,Random House,None,1,None,None,None,https://assetscdn.paytm.com/images/catalog/product/9/97/9780099474296_65496/0x1920/70/0.jpg,None,99474298,9780099474296,English,4244,You'll See It When You Believe It,272,1000,9780099474296_65496,31885378,February,2005,Random House,None,Bestseller Books,d5729c53b885fab0f487a47c1129fe9a,None,196


In [9]:
result = final_df.to_json(orient="index")
parsed = json.loads(result)

with open("final.json", 'w', encoding='utf8') as file:
    json.dump(parsed, file, indent=4) 

In [8]:
'''open_df = pd.read_csv('new_books.csv', dtype={'list_price': 'Int32', 'model': 'Int32', 'productcode': 'Int32', 'weight': 'Int32'})
open_df = open_df.astype(object).where(open_df.notna(), None)
print(type(open_df['list_price'][0]))
open_df'''

"open_df = pd.read_csv('new_books.csv', dtype={'list_price': 'Int32', 'model': 'Int32', 'productcode': 'Int32', 'weight': 'Int32'})\nopen_df = open_df.astype(object).where(open_df.notna(), None)\nprint(type(open_df['list_price'][0]))\nopen_df"